In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

In [ ]:
import keras_tuner as kt
from keras_tuner import RandomSearch
from GEH_LOSS import GEH_loss
import geh as ga

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error

In [ ]:
import relative_accuracy as ra
from statistics import mean, stdev

In [ ]:
# Load all Inputs and Output Data

# 5 mins (1 step ahead)
Deep_train_5   = np.load("Deep_train_5_multi.npz")['x'] 
Output_train_5 = np.load("Deep_train_5_multi.npz")['y']

Deep_test_5   = np.load("Deep_test_5_multi.npz")['x'] 
Output_test_5 = np.load("Deep_test_5_multi.npz")['y'] 

# 15 mins (3 steps ahead)
Deep_train_15   = np.load("Deep_train_15_multi.npz")['x'] 
Output_train_15 = np.load("Deep_train_15_multi.npz")['y']

Deep_test_15   = np.load("Deep_test_15_multi.npz")['x'] 
Output_test_15 = np.load("Deep_test_15_multi.npz")['y']

# 30 mins (6 steps ahead)
Deep_train_30   = np.load("Deep_train_30_multi.npz")['x'] 
Output_train_30 = np.load("Deep_train_30_multi.npz")['y']

Deep_test_30   = np.load("Deep_test_30_multi.npz")['x'] 
Output_test_30 = np.load("Deep_test_30_multi.npz")['y']

# 60 mins (12 steps ahead)
Deep_train_60   = np.load("Deep_train_60_multi.npz")['x'] 
Output_train_60 = np.load("Deep_train_60_multi.npz")['y']

Deep_test_60   = np.load("Deep_test_60_multi.npz")['x'] 
Output_test_60 = np.load("Deep_test_60_multi.npz")['y']

In [ ]:
# Wide Data
Xtrain = joblib.load("002weeks_train.save") 
Xtest = joblib.load("002weeks_test.save") 

In [ ]:
# 5 mins
# Delete first 15 samples
Wide_train_5 = np.delete(Xtrain, np.s_[0:15], 0)
Wide_test_5 = np.delete(Xtest, np.s_[0:15], 0)

In [ ]:
# 15 mins
# Delete first 17 samples
Wide_train_15 = np.delete(Xtrain, np.s_[0:17], 0)
Wide_test_15 = np.delete(Xtest, np.s_[0:17], 0)

In [ ]:
# 30 mins
# Delete first 20 samples
Wide_train_30 = np.delete(Xtrain, np.s_[0:20], 0)
Wide_test_30 = np.delete(Xtest, np.s_[0:20], 0)

In [ ]:
# 60 mins
# Delete first 27 samples
Wide_train_60 = np.delete(Xtrain, np.s_[0:26], 0)
Wide_test_60 = np.delete(Xtest, np.s_[0:26], 0)

In [ ]:
scaler_filename = "scaler.save"
scaler = joblib.load(scaler_filename) 

In [ ]:
Test      = pd.read_csv('01test_scaled.csv', index_col=None, parse_dates=[0])

# Reshape Samples

The first step is to split the input sequences into subsequences that can be processed by the CNN model. Here, each spatio-temporal sample can be split into three sub-samples, each with five time steps. The CNN can interpret each subsequence of five time steps and provide a time series of interpretations of the subsequences to the LSTM model to process as input.

In [ ]:
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 7                # No of loop detectors
n_seq = 3                     # Subsequences
n_steps = 5                   # time-step per subsequence
# val_percent = 0.12962       # 2 weeks
val_percent = 0.2         
batch_size = 32

#### 5 mins

In [ ]:
# Train
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xt5 = Deep_train_5.reshape((Deep_train_5.shape[0], n_seq, n_steps, n_features))
yt5 = scaler.inverse_transform(Output_train_5)

In [ ]:
# Test
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xv5 = Deep_test_5.reshape((Deep_test_5.shape[0], n_seq, n_steps, n_features))
yv5 = scaler.inverse_transform(Output_test_5)

#### 15 mins

In [ ]:
# Train
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xt15 = Deep_train_15.reshape((Deep_train_15.shape[0], n_seq, n_steps, n_features))
yt15 = scaler.inverse_transform(Output_train_15)

In [ ]:
# Test
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xv15 = Deep_test_15.reshape((Deep_test_15.shape[0], n_seq, n_steps, n_features))
yv15 = scaler.inverse_transform(Output_test_15)

#### 30 mins

In [ ]:
# Train
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xt30 = Deep_train_30.reshape((Deep_train_30.shape[0], n_seq, n_steps, n_features))
yt30 = scaler.inverse_transform(Output_train_30)

In [ ]:
# Test
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xv30 = Deep_test_30.reshape((Deep_test_30.shape[0], n_seq, n_steps, n_features))
yv30 = scaler.inverse_transform(Output_test_30)

#### 60 mins

In [ ]:
# Train
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xt60 = Deep_train_60.reshape((Deep_train_60.shape[0], n_seq, n_steps, n_features))
yt60 = scaler.inverse_transform(Output_train_60)

In [ ]:
# Test
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
Xv60 = Deep_test_60.reshape((Deep_test_60.shape[0], n_seq, n_steps, n_features))
yv60 = scaler.inverse_transform(Output_test_60)

### Reshaping the Data for Conv2D layer

For Conv2D, there is a need to add one more dimension to show we're dealing with 1 channel (since technically the images are in black and white, only showing values from 0-max flow on a single channel).

Conv1D - strides in 1 dimension
Conv2D - strides in 2 dimensions

In [ ]:
# define no_of_channels
n_channels = 1

In [ ]:
# Train data
Xt5  =  Xt5.reshape(Xt5.shape[0], n_seq, n_steps, n_features, n_channels)
Xt15 =  Xt15.reshape(Xt15.shape[0], n_seq, n_steps, n_features, n_channels)
Xt30 =  Xt30.reshape(Xt30.shape[0], n_seq, n_steps, n_features, n_channels)
Xt60 =  Xt60.reshape(Xt60.shape[0], n_seq, n_steps, n_features, n_channels)

# Validation data
Xv5  =  Xv5.reshape(Xv5.shape[0], n_seq, n_steps, n_features, n_channels)
Xv15 =  Xv15.reshape(Xv15.shape[0], n_seq, n_steps, n_features, n_channels)
Xv30 =  Xv30.reshape(Xv30.shape[0], n_seq, n_steps, n_features, n_channels)
Xv60 =  Xv60.reshape(Xv60.shape[0], n_seq, n_steps, n_features, n_channels)

In [ ]:
Wt5  = Wide_train_5
Wt15 = Wide_train_15
Wt30 = Wide_train_30
Wt60 = Wide_train_60

Wv5  = Wide_test_5
Wv15 = Wide_test_15
Wv30 = Wide_test_30
Wv60 = Wide_test_60

# CNN-LSTM model

In [ ]:
from tensorflow import keras
from keras_self_attention import SeqSelfAttention

# Early Stopping
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=5, restore_best_weights = True)

In [ ]:
Wide_train_5.shape

# 5 mins ahead

In [ ]:
def build_model_5(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wide_train_5.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt5.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    Wide_5 = keras.layers.Dense(units=units, activation="relu")(channel_Wide)
    Wide_5_exp = keras.layers.Reshape(target_shape=(1,-1))(Wide_5)
    
    
    # Deep Model
    # CNN 
    CNN_5 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_5 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_5)
    # LSTM          
    LSTM_5 = keras.layers.LSTM(units=units, activation='relu', return_sequences=True)(flatten_5)

    
    # Concatenation 
    concat = keras.layers.concatenate([Wide_5_exp, LSTM_5], axis=1)

    # Attention
    Att_5 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(Att_5)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_5 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_5.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss=GEH_loss, metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_5

In [ ]:
tuner5 = RandomSearch(
    build_model_5,
    objective="val_loss",
    max_trials=30,
    executions_per_trial=2,
    overwrite=True,
    directory=os.path.normpath('C:/Runs'),
    project_name="7m-CNN-LSTM-5",
)

In [ ]:
tuner5.search_space_summary()

In [ ]:
tuner5.search((Wt5,Xt5), yt5, epochs=200,
           validation_split = val_percent,
           callbacks = [early_stop],
           verbose=2)

In [ ]:
tuner5.results_summary(1)

### Best Model

In [ ]:
best_hp5 = tuner5.get_best_hyperparameters()[0]
model5 = tuner5.hypermodel.build(best_hp5)

In [ ]:
scaler_filename = "Best_HP/7m-ConvLSTM-5"
joblib.dump(best_hp5, scaler_filename) 

In [ ]:
model5.summary()

In [ ]:
model5.fit((Wt5,Xt5), yt5, 
           epochs = 100, 
           verbose = 1)

In [ ]:
model5.save("my_dir/7m-CNN-LSTM-5")

In [ ]:
metrics5 = pd.DataFrame(model5.history.history)

In [ ]:
metrics5[['loss']].plot()

In [ ]:
model5 = keras.models.load_model("my_dir/7m-CNN-LSTM-5", custom_objects={'GEH_loss': GEH_loss}) 

In [ ]:
model5.predict((Wv5,Xv5))

### Evaluation on inverse_transform

In [ ]:
# test_predictions5 = model5.predict((Wv5,Xv5))

# yv5 = scaler.inverse_transform(Test.drop('Timestamp', axis = 1))
# yv5 = yv5[15:,3]

# df = Test.drop(Test.head(15).index)
# df.reset_index(inplace = True)
# df = df.drop(columns=['index','Timestamp'])


# df['f4'] = pd.Series(test_predictions5.flatten())

# test_predictions5 = scaler.inverse_transform(df)

# test_predictions5 = test_predictions5[:,3]

# MSE5 = mean_squared_error(yv5,test_predictions5)

# MAE5 = mean_absolute_error(yv5,test_predictions5)

# RMSE5 = np.sqrt(MSE5)

# MAPE5 = mean_absolute_percentage_error(yv5,test_predictions5)*100

# eval5 = [MSE5, MAE5, RMSE5, MAPE5]
# print(eval5)

In [ ]:
# RA10 = ra.RA_10(np.asarray(yv5), np.asarray(test_predictions5))
# RA15 = ra.RA_15(np.asarray(yv5), np.asarray(test_predictions5))
# RA20 = ra.RA_20(np.asarray(yv5), np.asarray(test_predictions5))

# res5 = [RA10, RA15, RA20]
# print(res5)

# 15 mins ahead

In [ ]:
def build_model_15(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wide_train_15.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt15.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    Wide_15 = keras.layers.Dense(units=units, activation="relu")(channel_Wide)
    Wide_15_exp = keras.layers.Reshape(target_shape=(1,-1))(Wide_15)
    
    
    # Deep Model
    # CNN 
    CNN_15 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_15 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_15)
    # LSTM          
    LSTM_15 = keras.layers.LSTM(units=units, activation='relu', return_sequences=True)(flatten_15)

    
    # Concatenation 
    concat = keras.layers.concatenate([Wide_15_exp, LSTM_15], axis=1)

    # Attention
    Att_15 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(Att_15)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_15 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_15.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss=GEH_loss, metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
    
    return model_15

In [ ]:
tuner15 = RandomSearch(
    build_model_15,
    objective="val_loss",
    max_trials=30,
    executions_per_trial=2,
    overwrite=True,
    directory=os.path.normpath('C:/Runs'),
    project_name="7m-CNN-LSTM-15",
)

In [ ]:
tuner15.search_space_summary()

In [ ]:
tuner15.search((Wt15,Xt15), yt15, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop], 
            verbose=2)

In [ ]:
tuner15.results_summary(1)

### Best model

In [ ]:
best_hp15 = tuner15.get_best_hyperparameters()[0]
model15 = tuner15.hypermodel.build(best_hp15)

In [ ]:
scaler_filename = "Best_HP/7m-ConvLSTM-15"
joblib.dump(best_hp15, scaler_filename) 

In [ ]:
model15.summary()

In [ ]:
model15.fit((Wt15,Xt15), yt15, 
            epochs = 100, 
            verbose = 1)

In [ ]:
model15.save("my_dir/7m-CNN-LSTM-15")

In [ ]:
metrics15 = pd.DataFrame(model15.history.history)

In [ ]:
metrics15[['loss']].plot()

In [ ]:
model15 = keras.models.load_model("my_dir/7m-CNN-LSTM-15", custom_objects={'GEH_loss': GEH_loss})

In [ ]:
model15.predict((Wv15,Xv15))

### Evaluation on inverse_transform

In [ ]:
# test_predictions15 = model15.predict((Wv15,Xv15))

# yv15 = scaler.inverse_transform(Test.drop('Timestamp', axis = 1))
# yv15 = yv15[17:,3]

# df = Test.drop(Test.head(17).index)
# df.reset_index(inplace = True)
# df = df.drop(columns=['index','Timestamp'])
# df['f4'] = pd.Series(test_predictions15.flatten())

# test_predictions15 = scaler.inverse_transform(df)

# test_predictions15 = test_predictions15[:,3]

# MSE15 = mean_squared_error(yv15,test_predictions15)

# MAE15 = mean_absolute_error(yv15,test_predictions15)

# RMSE15 = np.sqrt(MSE15)

# MAPE15 = mean_absolute_percentage_error(yv15,test_predictions15)*100

# eval15 = [MSE15, MAE15, RMSE15, MAPE15]
# print(eval15)

In [ ]:
# RA10 = ra.RA_10(np.asarray(yv15), np.asarray(test_predictions15))
# RA15 = ra.RA_15(np.asarray(yv15), np.asarray(test_predictions15))
# RA20 = ra.RA_20(np.asarray(yv15), np.asarray(test_predictions15))

# res15 = [RA10, RA15, RA20]
# print(res15)

# 30 mins ahead

In [ ]:
def build_model_30(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wide_train_30.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt30.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    Wide_30 = keras.layers.Dense(units=units, activation="relu")(channel_Wide)
    Wide_30_exp = keras.layers.Reshape(target_shape=(1,-1))(Wide_30)
    
    
    # Deep Model
    # CNN 
    CNN_30 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_30 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_30)
    # LSTM          
    LSTM_30 = keras.layers.LSTM(units=units, activation='relu', return_sequences=True)(flatten_30)

    
    # Concatenation 
    concat = keras.layers.concatenate([Wide_30_exp, LSTM_30], axis=1)

    # Attention
    Att_30 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(Att_30)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_30 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_30.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss=GEH_loss, metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
    
    return model_30

In [ ]:
tuner30 = RandomSearch(
    build_model_30,
    objective="val_loss",
    max_trials=30,
    executions_per_trial=2,
    overwrite=True,
    directory=os.path.normpath('C:/Runs'),
    project_name="7m-CNN-LSTM-30",
)

In [ ]:
tuner30.search_space_summary()

In [ ]:
tuner30.search((Wt30,Xt30), yt30, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop],
            verbose=2)

In [ ]:
tuner30.results_summary(1)

### Best model

In [ ]:
best_hp30 = tuner30.get_best_hyperparameters()[0]
model30 = tuner30.hypermodel.build(best_hp30)

In [ ]:
scaler_filename = "7m-ConvLSTM-30"
joblib.dump(best_hp30, scaler_filename) 

In [ ]:
model30.summary()

In [ ]:
model30.fit((Wt30,Xt30), yt30, 
            epochs = 100, 
            verbose = 1)

In [ ]:
model30.save("my_dir/7m-CNN-LSTM-30")

In [ ]:
metrics30 = pd.DataFrame(model30.history.history)

In [ ]:
metrics30[['loss']].plot()

In [ ]:
model30 = keras.models.load_model("my_dir/7m-CNN-LSTM-30", custom_objects={'GEH_loss': GEH_loss})

In [ ]:
model30.predict((Wv30,Xv30))

### Evaluation on inverse_transform

In [ ]:
# test_predictions30 = model30.predict((Wv30,Xv30))

# yv30 = scaler.inverse_transform(Test.drop('Timestamp', axis = 1))
# yv30 = yv30[20:,3]

# df = Test.drop(Test.head(20).index)
# df.reset_index(inplace = True)
# df = df.drop(columns=['index','Timestamp'])
# df['f4'] = pd.Series(test_predictions30.flatten())

# test_predictions30 = scaler.inverse_transform(df)

# test_predictions30 = test_predictions30[:,3]

# MSE30 = mean_squared_error(yv30,test_predictions30)

# MAE30 = mean_absolute_error(yv30,test_predictions30)

# RMSE30 = np.sqrt(MSE30)

# MAPE30 = mean_absolute_percentage_error(yv30,test_predictions30)*100

# eval30 = [MSE30, MAE30, RMSE30, MAPE30]
# print(eval30)

In [ ]:
# RA10 = ra.RA_10(np.asarray(yv30), np.asarray(test_predictions30))
# RA15 = ra.RA_15(np.asarray(yv30), np.asarray(test_predictions30))
# RA20 = ra.RA_20(np.asarray(yv30), np.asarray(test_predictions30))

# res30 = [RA10, RA15, RA20]
# print(res30)

# 60 mins ahead

In [ ]:
def build_model_60(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wide_train_60.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt60.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    Wide_60 = keras.layers.Dense(units=units, activation="relu")(channel_Wide)
    Wide_60_exp = keras.layers.Reshape(target_shape=(1,-1))(Wide_60)
    
    
    # Deep Model
    # CNN 
    CNN_60 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_60 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_60)
    # LSTM          
    LSTM_60 = keras.layers.LSTM(units=units, activation='relu', return_sequences=True)(flatten_60)

    
    # Concatenation 
    concat = keras.layers.concatenate([Wide_60_exp, LSTM_60], axis=1)

    # Attention
    Att_60 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(Att_60)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_60 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_60.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss=GEH_loss, metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_60

In [ ]:
tuner60 = RandomSearch(
    build_model_60,
    objective="val_loss",
    max_trials=30,
    executions_per_trial=2,
    overwrite=True,
    directory=os.path.normpath('C:/Runs'),
    project_name="7m-CNN-LSTM-60",
)

In [ ]:
tuner60.search_space_summary()

In [ ]:
tuner60.search((Wt60,Xt60), yt60, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop],
            verbose=2)

In [ ]:
tuner60.results_summary(1)

### Best model

In [ ]:
best_hp60 = tuner60.get_best_hyperparameters()[0]
model60 = tuner60.hypermodel.build(best_hp60)

In [ ]:
scaler_filename = "Best_HP/7m-ConvLSTM-60"
joblib.dump(best_hp60, scaler_filename) 

In [ ]:
model60.summary()

In [ ]:
model60.fit((Wt60,Xt60), yt60, 
            epochs = 100, 
            verbose = 1
            )

In [ ]:
model60.save("my_dir/7m-CNN-LSTM-60")

In [ ]:
metrics60 = pd.DataFrame(model60.history.history)

In [ ]:
metrics60[['loss']].plot()

In [ ]:
model60 = keras.models.load_model("my_dir/7m-CNN-LSTM-60", custom_objects={'GEH_loss': GEH_loss})

In [ ]:
model60.predict((Wv60,Xv60))

### Evaluation on inverse_transform

In [ ]:
# test_predictions60 = model60.predict((Wv60,Xv60))

# yv60 = scaler.inverse_transform(Test.drop('Timestamp', axis = 1))
# yv60 = yv60[26:,3]

# df = Test.drop(Test.head(26).index)
# df.reset_index(inplace = True)
# df = df.drop(columns=['index','Timestamp'])
# df['f4'] = pd.Series(test_predictions60.flatten())

# test_predictions60 = scaler.inverse_transform(df)

# test_predictions60 = test_predictions60[:,3]

# MSE60 = mean_squared_error(yv60,test_predictions60)

# MAE60 = mean_absolute_error(yv60,test_predictions60)

# RMSE60 = np.sqrt(MSE60)

# MAPE60 = mean_absolute_percentage_error(yv60,test_predictions60)*100

# eval60 = [MSE60, MAE60, RMSE60, MAPE60]
# print(eval60)

In [ ]:
# RA10 = ra.RA_10(np.asarray(yv60), np.asarray(test_predictions60))
# RA15 = ra.RA_15(np.asarray(yv60), np.asarray(test_predictions60))
# RA20 = ra.RA_20(np.asarray(yv60), np.asarray(test_predictions60))

# res60 = [RA10, RA15, RA20]
# print(res60)

# RESULTS

## General results

In [ ]:
l = ['MSE','MAE','RMSE','MAPE','Horizon']

In [ ]:
eval5.append(5)
eval15.append(15)
eval30.append(30)
eval60.append(60)

In [ ]:
df = pd.DataFrame(np.array([eval5,eval15,eval30,eval60]),columns=l)
df.set_index('Horizon')

## Relative accuracy

In [ ]:
m = ['10%','15%','20%','Horizon']

In [ ]:
res5.append(5)
res15.append(15)
res30.append(30)
res60.append(60)

In [ ]:
df1 = pd.DataFrame(np.array([res5,res15,res30,res60]),columns=m)
df1.set_index('Horizon')